# Multilayer Perception (MLP)

- ### Step 1: Import Necessary Libraries
  ##### We import all the necessary libraries for data manipulation, model building, and hyperparameter tuning.

In [1]:
# Import necessary libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
import kerastuner as kt
from tensorflow import keras


C:\Users\DELL\AppData\Local\Temp\ipykernel_8200\3464076288.py:10: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner as kt


- ### Step 2: Load and Preprocess Data
  ##### We load the dataset, preprocess the RGB columns, encode the labels, split the data into training and testing sets, and standardize the features.

In [2]:
#Initial form of the data 
data = pd.read_csv('demmo_data.csv')

data.head()

,Emotional_Word,RGB_1,RGB_2,RGB_3
0,Cute,"[251, 167, 157]","[255, 242, 124]","[179, 22, 61]"
1,Childlike,"[251, 103, 89]","[255, 242, 124]","[153, 216, 212]"
2,Pretty,"[251, 167, 157]","[255, 242, 63]","[78, 181, 135]"
3,Sweet,"[251, 103, 89]","[253, 192, 145]","[251, 174, 193]"
4,Amusing,"[253, 166, 74]","[140, 201, 25]","[90, 177, 132]"


In [3]:
# Preprocess RGB columns
data['RGB_1'] = data['RGB_1'].apply(lambda x: eval(x))
data['RGB_2'] = data['RGB_2'].apply(lambda x: eval(x))
data['RGB_3'] = data['RGB_3'].apply(lambda x: eval(x))

# Extract features and labels
X = np.hstack([np.vstack(data['RGB_1']), np.vstack(data['RGB_2']), np.vstack(data['RGB_3'])])
y = data['Emotional_Word']

data.head()

,Emotional_Word,RGB_1,RGB_2,RGB_3
0,Cute,"[251, 167, 157]","[255, 242, 124]","[179, 22, 61]"
1,Childlike,"[251, 103, 89]","[255, 242, 124]","[153, 216, 212]"
2,Pretty,"[251, 167, 157]","[255, 242, 63]","[78, 181, 135]"
3,Sweet,"[251, 103, 89]","[253, 192, 145]","[251, 174, 193]"
4,Amusing,"[253, 166, 74]","[140, 201, 25]","[90, 177, 132]"


- ### Step 3: Encode Labels and Split Data
  ##### Encode the labels, split the data into training and testing sets, and standardize the features.

In [4]:
# Extract features and labels
X = np.hstack([np.vstack(data['RGB_1']), np.vstack(data['RGB_2']), np.vstack(data['RGB_3'])])
y = data['Emotional_Word']

# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
y_categorical = to_categorical(y_encoded)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y_categorical, test_size=0.2, random_state=42)

# Standardize data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Print first 5 lines of encoded labels
print("First 5 lines of encoded labels:", y_encoded[:5])

# Print first 5 lines of standardized features
print("First 5 lines of standardized training features:\n", X_train[:5])

First 5 lines of encoded labels: [ 32  25 104 140   4]
First 5 lines of standardized training features:
 [[ 0.81555294 -0.45688637 -0.24194424  0.90616419 -0.13704796 -1.08054255
  -0.18318621  0.76720268  1.82060601]
 [-1.39819398 -1.40484878 -1.01887779 -0.38462758 -0.68804575 -0.36056995
  -1.57020784 -0.72564359 -0.56371856]
 [ 0.81555294 -0.38396618  1.00343455 -0.41560658  1.40843363 -1.08054255
   1.04972191 -1.54670903  0.60483655]
 [-1.96806942  0.09001503  0.11224606 -0.6634386   0.60209541  1.07937523
  -1.30307775 -0.96201091  0.69926525]
 [-1.97902857  1.43903847  1.86034655  0.90616419 -1.09121486 -1.08054255
   1.04972191 -0.48927626  0.99435492]]


- ### Step 4: Define the Model Building Function for Keras Tuner (MLP)
  ##### This function defines the MLP model and includes hyperparameters for tuning the number of units, dropout rate, number of layers, and learning rate.

In [5]:
def build_model(hp):
    model = Sequential()
    model.add(Input(shape=(9,)))
    model.add(Dense(units=hp.Int('units', min_value=32, max_value=512, step=64), activation='relu', kernel_regularizer=l2(1e-4)))
    model.add(BatchNormalization())
    model.add(Dropout(hp.Float('dropout', 0, 0.5, step=0.1)))
    
    for i in range(hp.Int('num_layers', 1, 5)):  # Increase the potential number of layers
        model.add(Dense(units=hp.Int('units_' + str(i), min_value=32, max_value=512, step=64), activation='relu', kernel_regularizer=l2(1e-4)))
        model.add(BatchNormalization())
        model.add(Dropout(hp.Float('dropout_' + str(i), 0, 0.5, step=0.1)))
    
    model.add(Dense(len(label_encoder.classes_), activation='softmax'))
    
    model.compile(optimizer=SGD(learning_rate=hp.Choice('learning_rate', values=[1e-1, 1e-2, 1e-3]), momentum=hp.Float('momentum', 0.5, 0.9, step=0.1)),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model


- ### Step  5: Initialize the Keras Tuner and Search for the Best Hyperparameters
  ##### We initialize the Keras Tuner and search for the best hyperparameters using the training and validation data.

In [6]:
tuner = kt.Hyperband(
    build_model,
    objective='val_accuracy',
    max_epochs=50,
    factor=3,
    directory='my_dir',
    project_name='emotion_prediction'
)

tuner.search(
    X_train,
    y_train,
    epochs=100,
    validation_data=(X_test, y_test),
    callbacks=[keras.callbacks.EarlyStopping(patience=10)]  # Adjust patience for early stopping
)


Reloading Tuner from my_dir\emotion_prediction\tuner0.json


- ### Step 6: Retrieve the Best Model and Train it
  ##### We retrieve the best model found by the Keras Tuner and train it using the training data.

In [7]:
# Retrieve the best model
best_model = tuner.get_best_models(num_models=1)[0]

# Train the best model
history = best_model.fit(
    X_train,  # Training features
    y_train,  # Training labels
    epochs=50,  # Number of epochs for training
    validation_data=(X_test, y_test),  # Validation data for evaluation
    callbacks=[keras.callbacks.EarlyStopping(patience=5)]  # Early stopping to prevent overfitting
)

NameError: name 'Input' is not defined

In [ ]:
# Print model summary
best_model.summary()

# Print final training accuracy
print(f'Final Training Accuracy: {history.history["accuracy"][-1]}')
print(f'Final Validation Accuracy: {history.history["val_accuracy"][-1]}')

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 448)            │         4,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 448)            │         1,792 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 448)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 512)            │       229,888 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 512)            │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 480)            │       123,360 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 480)            │         1,920 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 480)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 155)            │        74,555 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,137,400 (4.34 MB)

 Trainable params: 567,003 (2.16 MB)

 Non-trainable params: 3,392 (13.25 KB)

 Optimizer params: 567,005 (2.16 MB)

Final Training Accuracy: 0.07692307978868484
Final Validation Accuracy: 0.0


- ### Step 7: Evaluate the Model
  ##### We evaluate the trained model on the test data and print the test accuracy.

In [ ]:
# Evaluate the model
loss, accuracy = best_model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy}')

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.0000e+00 - loss: 5.1646
Test Accuracy: 0.0


- ### Step 8: Save and Load the Model
  ##### Save the trained model to a file for future use, and show how to load it back when needed.

 - ### Step 9: Make Predictions
   #####  Define a function to predict the emotional word based on new RGB values by processing the input and using the trained model.

In [ ]:
# Function to predict emotional word based on new RGB values
def predict_emotion(rgb1, rgb2, rgb3):
    rgb_values = np.hstack([rgb1, rgb2, rgb3]).reshape(1, -1)
    rgb_values = scaler.transform(rgb_values)
    prediction = best_model.predict(rgb_values)
    predicted_label = label_encoder.inverse_transform([np.argmax(prediction)])
    return predicted_label[0]

# Example prediction
new_rgb1 = [251, 167, 157]
new_rgb2 = [255, 242, 124]
new_rgb3 = [179, 22, 61]
print("Predicted Emotion:", predict_emotion(new_rgb1, new_rgb2, new_rgb3))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step
Predicted Emotion: Cute
